In [ ]:
import os

import datasets as datasets

from nir.training.train_sentence_encoder import train_huggingface_transformer

TASK_NAME = "contrastive_passage"
OUTPUT_DIR = f"./outputs/cross_encoder/{TASK_NAME}/"
DATA_DIR = "../data/"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
from training.losses.contrastive import SupConLoss
from datasets import DatasetDict
from evaluation.cross_validation import CrossValidator
from transformers import AutoTokenizer
import datasets

tokenizer = AutoTokenizer.from_pretrained("squeezebert-base-headless")

# 5 Fold cross validation

vals_ds = datasets.load_dataset('csv', [
    datasets.ReadInstruction('train', from_=k, to=k+20, unit='%')
    for k in range(0, 100, 20)], data_files={"train": "med_marco_train_enriched.csv"})
trains_ds = datasets.load_dataset('bookcorpus', [
    (datasets.ReadInstruction('train', to=k, unit='%') +
    datasets.ReadInstruction('train', from_=k+20, unit='%'))
    for k in range(0, 100, 20)], data_files={"train": "med_marco_train_enriched.csv"})

dataset = DatasetDict({'train': trains_ds, "val": vals_ds})

trainer = train_huggingface_transformer("squeezebert-base-headless",
                              tokenizer,
                              OUTPUT_DIR,
                              None,
                              None,
                              dataset,
                              loss_func=SupConLoss())